In [1]:
import pandas as pd
import numpy as np
import nltk
import gensim
import string

In [2]:
test = pd.read_csv("test_with_scraping.csv")
train = pd.read_csv("train_with_scraping.csv")

## 加载垃圾词

In [3]:
import pickle
fr = open("garbage.txt", 'rb')
garbage = pickle.load(fr)
fr.close()

In [4]:
garbage = [t[0] for t in garbage]

## 在训练集探索特征

In [5]:
train['Event ID'] = train['Event ID'].fillna(0)
train.loc[train["Abstract"].isnull(), 'Abstract'] = train[train["Abstract"].isnull()]['Title']
train.loc[train['in_reply_to_status_text'].notnull(), 'Abstract'] = train[train['in_reply_to_status_text'].notnull()]['in_reply_to_status_text']

In [6]:
def getTokens(all_command):
    handle_all_command = []
    lines = all_command.tolist()
    stopwords = set(nltk.corpus.stopwords.words("english"))
    table = str.maketrans("","",string.punctuation)
    for line in lines:
        line = " ".join([word for word in str(line).split(" ") if '@' not in word and 'http' not in word and "RT" not in word])
        tokens = nltk.word_tokenize(line)
        tokens = [w.lower() for w in tokens]
        tokens = [w.translate(table) for w in tokens]
        tokens = [w for w in tokens if w.isalpha()]
        tokens = [w for w in tokens if w not in stopwords and w not in garbage]
        handle_all_command.append(tokens)
    return handle_all_command

In [7]:
%time train_token = getTokens(train.Abstract)

CPU times: user 20.2 s, sys: 49.7 ms, total: 20.3 s
Wall time: 20.4 s


In [8]:
train['len'] = [len(l) for l in train_token]
train = train[train['len'] > 3]

## 处理好train后，进行explore

### 判断问句

In [9]:
ab_y0 = train[train['Event ID'] == 0]['Abstract']
ab_y1 = train[train['Event ID'] > 0]['Abstract']

In [10]:
y1_withMark = sum([1 for text in ab_y1 if "?" in text])
y0_withMark = sum([1 for text in ab_y0 if "?" in text])
print("食物相关带问号比例：", y1_withMark * 100 / len(ab_y1), "%")
print("非食物相关带问号比例：", y0_withMark * 100 / len(ab_y0), "%")

食物相关带问号比例： 4.480164158686731 %
非食物相关带问号比例： 22.17391304347826 %


### 查看首个单词字母小写的影响

In [11]:
def counter(all_command):
    res=[]
    lines = all_command.tolist()
    table = str.maketrans("","",string.punctuation)
    for line in lines:
        line = " ".join([word for word in str(line).split(" ") if '@' not in word and 'http' not in word and "RT" not in word])
        tokens = nltk.word_tokenize(line)
        tokens = [w.translate(table) for w in tokens]
        tokens = [w for w in tokens if w.isalpha()]
        res.append(True if tokens[0][0].islower() else False)
    return res


In [12]:
%time train['isLower'] = counter(train['Abstract'])
len(train[train['isLower']])

CPU times: user 1.16 s, sys: 4.55 ms, total: 1.17 s
Wall time: 1.17 s


214

In [13]:
lower_y0 = train[train['Event ID'] == 0]['isLower']
lower_y1 = train[train['Event ID'] > 0]['isLower']

In [14]:
y1_isLower = sum([1 for x in lower_y1 if x == True])
y0_isLower = sum([1 for x in lower_y0 if x == True])
print("食物相关首字母小写比例：", y1_isLower * 100 / len(lower_y0), "%")
print("非食物相关首字母小写比例：", y0_isLower * 100 / len(lower_y1), "%")

食物相关首字母小写比例： 3.3043478260869565 %
非食物相关首字母小写比例： 6.019151846785226 %


## 处理测试集

In [15]:
%time test_token = getTokens(test.Abstract)
test['len'] = [len(l) for l in test_token]

CPU times: user 1min 8s, sys: 184 ms, total: 1min 8s
Wall time: 1min 8s


In [16]:
test = test[test['len'] > 3]

In [17]:
sum([1 for text in test.Abstract.astype(str) if "?" in text])

3664